In [18]:
import cv2
import numpy as np

In [22]:
def rgb_2_yuv(rgb):
    rgb_xyz = np.array([[0.4124, 0.3576, 0.1805],
                        [0.2126, 0.7152, 0.0722],
                        [0.0193, 0.1192, 0.9505]])
    print(rgb_xyz.shape)
    xyz = np.einsum('ij,jk->ik', rgb_xyz, rgb)

    d = xyz[0] + 15 * xyz[1] + 3 * xyz[2]
    u = 4 * xyz[0] / d
    v = 9 * xyz[1] / d
    return np.stack((xyz[1], u, v))


def color_diff_yuv(ref, test):
    Auv = rgb_2_yuv(ref)
    A2uv = rgb_2_yuv(test)
    duv = A2uv[1:] - Auv[1:]
    # color difference (as jnd)
    duv2 = np.sqrt(np.sum(np.square(duv), axis=0)) / 0.077
    # relative luminance difference
    alum = np.average((A2uv[0], Auv[0]), axis=0)
    mask = alum > 0
    dl = A2uv[0] - Auv[0]
    # as jnd
    dl[mask] = dl[mask]/alum[mask] / 0.003

    # Relative Luminance Difference (dl) + Chrominance Difference (duv2) + Final Color Difference Metric
    return np.sqrt(np.sum(np.stack((dl, duv2)), axis=0))

In [44]:
ref_path = "gt_hdr_b_1_1.hdr"
test_path = "generated_hdr_b_1_1.hdr"

In [45]:
# IMREAD_ANYDEPTH is needed because even though the data is stored in 8-bit channels
# when it's read into memory it's represented at a higher bit depth
img_ref = cv2.imread(ref_path, flags=cv2.IMREAD_ANYDEPTH)
h, w, channels = img_ref.shape
img_ref = img_ref.reshape(h*w, channels)
print(img_ref.shape)

(3276800, 3)


In [46]:
img_test = cv2.imread(test_path, flags=cv2.IMREAD_ANYDEPTH)
print(img_test.shape)
h, w, channels = img_test.shape
img_test = img_test.reshape(h*w, channels)
print(img_test.shape)

(1280, 2560, 3)
(3276800, 3)


In summary, the return value represents a comprehensive color difference metric that considers both luminance and chrominance components for assessing the perceptual dissimilarity between two RGB images.

In [47]:
value = color_diff_yuv(img_ref, img_test)
value

(3, 3)
(3, 3)


array([24.6808409 , 24.7797112 , 24.72708538])